In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
#  Create a test graph
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_pl
                                        acement=True))
# Runs the op.
print(sess.run(c))30°21°	Sun
29°22°	Mon


[[ 22.  28.]
 [ 49.  64.]]


Exercise 8
===

In [15]:
# We will first train the model to identify digits 0 to 4 and then do transfer training to identify images 5 to 9.

def divide_dataset_into_two_sets(images, labels):
    assert len(images) == len(labels)
    zero_to_four_images = []
    zero_to_four_labels = []
    five_to_nine_images = []
    five_to_nine_labels = []
    four_label = 4
    for i in range(len(images)):
        if labels[i] <= four_label:
            zero_to_four_images.append(images[i])
            zero_to_four_labels.append(labels[i])
        else:
            five_to_nine_images.append(images[i])
            five_to_nine_labels.append(labels[i])
    return np.array(zero_to_four_images), np.array(zero_to_four_labels), np.array(five_to_nine_images), np.array(five_to_nine_labels)

train_zero_to_four_images, train_zero_to_four_labels, train_five_to_nine_images, train_five_to_nine_labels = divide_dataset_into_two_sets(mnist.train.images, mnist.train.labels)
validation_zero_to_four_images, validation_zero_to_four_labels, validation_five_to_nine_images, validation_five_to_nine_labels = divide_dataset_into_two_sets(mnist.validation.images, mnist.validation.labels)
test_zero_to_four_images, test_zero_to_four_labels, test_five_to_nine_images, test_five_to_nine_labels = divide_dataset_into_two_sets(mnist.test.images, mnist.test.labels)

In [4]:
print(test_zero_to_four_images.shape)
print(test_zero_to_four_labels.shape)
print(test_five_to_nine_images.shape)
print(test_five_to_nine_labels.shape)

(5139, 784)
(5139,)
(4861, 784)
(4861,)


In [5]:
number_of_inputs = 28 * 28
n_hidden_per_layer = [100, 100, 100, 100, 100]
n_output = 5

def he_normal_initialisation(n_inputs, n_outputs):
    stddev = np.power(2 / (n_inputs + n_outputs), 1 / np.sqrt(2))
    # truncated normal distributions limit the size of the weights, speeding up the training time.
    return tf.truncated_normal((n_inputs, n_outputs), stddev=stddev)

def he_uniform_initialisation(n_inputs, n_outputs):
    r = np.power(6 / (n_inputs + n_outputs), 1 / np.sqrt(2))
    # truncated normal distributions limit the size of the weights, speeding up the training time.
    return tf.random_uniform((n_inputs, n_outputs), -r, r)

def neuron_layer(X, n_neurons, name):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        W = tf.Variable(he_normal_initialisation(n_inputs, n_neurons), name="weights")
        b = tf.Variable(tf.zeros([n_neurons]), name="biases")
        z = tf.matmul(X, W) + b
        return tf.nn.elu(z)

with tf.device("/gpu:0"):
    x = tf.placeholder(tf.float32, shape=(None, number_of_inputs), name="input")
    y = tf.placeholder(tf.int64, shape=(None), name="y")

    with tf.name_scope("dnn"):
        input_tensor = x
        for i in range(len(n_hidden_per_layer)):
            input_tensor = neuron_layer(input_tensor, n_hidden_per_layer[i], "hidden" + str(i + 1))
        logits = neuron_layer(input_tensor, n_output, "output")

    with tf.name_scope("loss"):
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
        loss = tf.reduce_mean(cross_entropy, name="loss")

    with tf.name_scope("training"):
        optimizer = tf.train.AdamOptimizer()
        training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    k = 1
    correctness = tf.nn.in_top_k(logits, y, k)
    accuracy = tf.reduce_mean(tf.cast(correctness, tf.float32)) * 100

In [6]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

interim_checkpoint_path = "./checkpoints/mnist_model.ckpt"
early_stopping_checkpoint_path = "./checkpoints/mnist_model_early_stopping.ckpt"

from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
log_dir = "{}/run-{}/".format(root_logdir, now)

loss_summary = tf.summary.scalar('loss', loss)
accuracy_summary = tf.summary.scalar("accuracy", accuracy)
summary_op = tf.summary.merge([loss_summary, accuracy_summary])
file_writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())

In [7]:
batch_size = 200
        
def create_next_batch_fn(images, labels, batch_size):
    assert len(images) == len(labels)
    current_batch = 0
    def next_batch():
        nonlocal current_batch
        i = current_batch
        #print(current_batch)
        current_batch = (current_batch + batch_size) % len(images)
        return images[i:i+batch_size], labels[i:i+batch_size]
    return next_batch

In [11]:
next_test_batch = create_next_batch_fn(test_zero_to_four_images, test_zero_to_four_labels, batch_size)

In [12]:
print(next_test_batch()[1])

[2 1 0 4 1 4 0 0 1 3 4 4 0 4 0 1 3 1 3 4 2 1 2 1 1 4 2 3 1 2 4 4 3 0 4 1 3
 4 4 3 0 0 2 1 3 2 2 4 3 1 3 3 1 4 1 0 4 2 1 4 3 4 4 4 2 4 0 1 0 1 4 3 1 1
 2 0 2 1 0 3 4 4 4 4 1 4 4 2 3 2 1 1 1 0 2 0 1 1 1 0 0 3 1 4 2 3 1 1 1 3 2
 4 3 0 3 2 2 1 2 4 1 3 3 2 2 4 1 2 3 0 4 4 2 4 1 2 2 1 1 0 3 0 1 4 1 2 1 2
 2 4 1 2 2 0 4 0 0 2 4 1 2 4 0 2 4 3 3 0 0 3 1 2 2 3 0 4 2 0 1 1 2 1 3 3 1
 3 1 0 1 3 1 1 1 4 2 2 0 3 2 0]


In [ ]:
epochs = 1
n_batches = int(np.ceil(mnist.train.num_examples // batch_size))

early_stopping_check_frequency = batch_size // 4
early_stopping_check_limit = batch_size * 2

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(init)
    #saver.restore(sess, interim_checkpoint_path)
    
    best_validation_acc = 0.0
    best_validation_step = 0
    for epoch in range(epochs):
        print("epoch", epoch)
        for batch_index in range(n_batches):
            step = epoch * n_batches + batch_index
            X_batch, y_batch = next_test_batch()
            if batch_index % 10 == 0:
                summary_str = summary_op.eval(feed_dict={x: X_batch, y: y_batch})
                file_writer.add_summary(summary_str, step)
            t, l, a = sess.run([training_op, loss, accuracy], feed_dict={x: X_batch, y: y_batch})
            if batch_index % 10 == 0: print("loss:", l, "training accuracy:", a)
            # Early stopping check
            if batch_index % early_stopping_check_frequency == 0:
                validation_acc = accuracy.eval(feed_dict={x: validation_zero_to_four_images, y: validation_zero_to_four_labels})
                print("validation accuracy", validation_acc)
                if validation_acc > best_validation_acc:
                    saver.save(sess, early_stopping_checkpoint_path)
                    best_validation_acc = validation_acc
                    best_validation_step = step
                elif step >= (best_validation_step + early_stopping_check_limit):
                    print("Stopping early during epoch", epoch)
                    break
        else:
            continue
        break
    save_path = saver.save(sess, interim_checkpoint_path)
    test_acc = accuracy.eval(feed_dict={x: test_zero_to_four_images, y: test_zero_to_four_labels})
    print(">>>>>>>>>> test dataset accuracy:", test_acc)

    save_path = saver.save(sess, "./checkpoints/mnist_model_final.ckpt")

epoch 0
loss: 1.60959 training accuracy: 21.5
validation accuracy 39.6403
loss: 1.37695 training accuracy: 64.5
loss: 0.787918 training accuracy: 71.5
loss: 0.532191 training accuracy: 82.0
loss: 0.173437 training accuracy: 95.0
loss: 0.109017 training accuracy: 96.5
validation accuracy 93.6278
loss: 0.112182 training accuracy: 96.5
loss: 0.110039 training accuracy: 96.5
loss: 0.0441707 training accuracy: 98.5
loss: 0.1036 training accuracy: 97.5
loss: 0.0552331 training accuracy: 98.5
validation accuracy 95.5825
loss: 0.0486561 training accuracy: 99.0
loss: 0.0987659 training accuracy: 97.5
loss: 0.0111191 training accuracy: 99.5
loss: 0.0429868 training accuracy: 97.5
loss: 0.0456751 training accuracy: 96.5
validation accuracy 95.6998


In [33]:
print(dir(mnist))
print(mnist.train.num_examples)
print(mnist.validation.num_examples)
print(mnist.test.num_examples)

['__add__', '__class__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmul__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '_asdict', '_fields', '_make', '_replace', '_source', 'count', 'index', 'test', 'train', 'validation']
55000
5000
10000


Exercise 9
===

In [ ]:
number_of_inputs = 28 * 28
n_hidden_per_layer = [100, 100, 100, 100, 100]
n_output = 10

def he_normal_initialisation(n_inputs, n_outputs):
    stddev = np.power(2 / (n_inputs + n_outputs), 1 / np.sqrt(2))
    # truncated normal distributions limit the size of the weights, speeding up the training time.
    return tf.truncated_normal((n_inputs, n_outputs), stddev=stddev)

def he_uniform_initialisation(n_inputs, n_outputs):
    r = np.power(6 / (n_inputs + n_outputs), 1 / np.sqrt(2))
    # truncated normal distributions limit the size of the weights, speeding up the training time.
    return tf.random_uniform((n_inputs, n_outputs), -r, r)

def neuron_layer(X, n_neurons, name):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        W = tf.Variable(he_normal_initialisation(n_inputs, n_neurons), name="weights")
        b = tf.Variable(tf.zeros([n_neurons]), name="biases")
        z = tf.matmul(X, W) + b
        return tf.nn.elu(z)

with tf.device("/gpu:0"):
    x = tf.placeholder(tf.float32, shape=(None, number_of_inputs), name="input")
    y = tf.placeholder(tf.int64, shape=(None), name="y")

    with tf.name_scope("dnn"):
        input_tensor = x
        for i in range(len(n_hidden_per_layer)):
            input_tensor = neuron_layer(input_tensor, n_hidden_per_layer[i], "hidden" + str(i + 1))
        logits = neuron_layer(input_tensor, n_output, "output")

    with tf.name_scope("loss"):
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
        loss = tf.reduce_mean(cross_entropy, name="loss")

    with tf.name_scope("training"):
        optimizer = tf.train.AdamOptimizer()
        training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    k = 1
    correctness = tf.nn.in_top_k(logits, y, k)
    accuracy = tf.reduce_mean(tf.cast(correctness, tf.float32)) * 100

In [ ]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

interim_checkpoint_path = "./checkpoints/mnist_model.ckpt"
early_stopping_checkpoint_path = "./checkpoints/mnist_model_early_stopping.ckpt"

from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
log_dir = "{}/run-{}/".format(root_logdir, now)

loss_summary = tf.summary.scalar('loss', loss)
accuracy_summary = tf.summary.scalar("accuracy", accuracy)
summary_op = tf.summary.merge([loss_summary, accuracy_summary])
file_writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())

In [ ]:
epochs = 20
batch_size = 200
n_batches = int(np.ceil(mnist.train.num_examples // batch_size))

early_stopping_check_frequency = batch_size // 4
early_stopping_check_limit = batch_size * 2

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(init)
    #saver.restore(sess, interim_checkpoint_path)
    
    best_validation_acc = 0.0
    best_validation_step = 0
    for epoch in range(epochs):
        print("epoch", epoch)
        for batch_index in range(n_batches):
            step = epoch * n_batches + batch_index
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            if batch_index % 10 == 0:
                summary_str = summary_op.eval(feed_dict={x: X_batch, y: y_batch})
                file_writer.add_summary(summary_str, step)
            t, l, a = sess.run([training_op, loss, accuracy], feed_dict={x: X_batch, y: y_batch})
            if batch_index % 10 == 0: print("loss:", l, "test accuracy:", a)
            # Early stopping check
            if batch_index % early_stopping_check_frequency == 0:
                validation_acc = accuracy.eval(feed_dict={x: mnist.validation.images, y: mnist.validation.labels})
                print("validation accuracy", validation_acc)
                if validation_acc > best_validation_acc:
                    saver.save(sess, early_stopping_checkpoint_path)
                    best_validation_acc = validation_acc
                    best_validation_step = step
                elif step >= (best_validation_step + early_stopping_check_limit):
                    print("Stopping early during epoch", epoch)
                    break
        else:
            continue
        break
    save_path = saver.save(sess, interim_checkpoint_path)
    test_acc = accuracy.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels})
    print(">>>>>>>>>> test dataset accuracy:", test_acc)

    save_path = saver.save(sess, "./checkpoints/mnist_model_final.ckpt")